In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'My Drive'


In [ ]:
!curl ipecho.net/plain

35.196.85.206

In [ ]:
pip install keras-video-generators

In [ ]:
%cd /content/gdrive/My\ Drive/Thesis/MovieLens20M

/content/gdrive/My Drive/Thesis/MovieLens20M


In [ ]:
pip uninstall keras-preprocessing

Uninstalling Keras-Preprocessing-1.1.2:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras_Preprocessing-1.1.2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/keras_preprocessing/*
Proceed (y/n)? y
  Successfully uninstalled Keras-Preprocessing-1.1.2


In [ ]:
pip install git+https://github.com/keras-team/keras-preprocessing.git

  Cloning https://github.com/keras-team/keras-preprocessing.git to /tmp/pip-req-build-jq74khw3
  Running command git clone -q https://github.com/keras-team/keras-preprocessing.git /tmp/pip-req-build-jq74khw3
  Created wheel for Keras-Preprocessing: filename=Keras_Preprocessing-1.1.2-cp36-none-any.whl size=43005 sha256=eb37160334331e2080657a4cf953b2c0df81e7cc57e0ce577d3ba8570be7503a
  Stored in directory: /tmp/pip-ephem-wheel-cache-uzqcs6ht/wheels/03/a0/39/171f6040d36f36c71168dc69afa81334351b20955dc36ce932
Successfully built Keras-Preprocessing


In [ ]:
import glob
import os
from keras.models import Sequential
"""Import from keras_preprocessing not from keras.preprocessing, because Keras may or maynot contain the features discussed here depending upon when you read this article, until the keras_preprocessed library is updated in Keras use the github version."""
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint

In [ ]:
df = pd.read_csv("current_used_data.csv")
df["Genres"]=df["Genres"].apply(lambda x:x.strip('][').replace('\'','').split(","))
print(df)

       Unnamed: 0     ID_Frame                       Genres
0               0    757_0.png                      [Drama]
1               1    757_1.png                      [Drama]
2               2    757_2.png                      [Drama]
3               3    757_3.png                      [Drama]
4               4    757_4.png                      [Drama]
...           ...          ...                          ...
71183       71183   1014_9.png  [Children,  Comedy,  Drama]
71184       71184  1014_10.png  [Children,  Comedy,  Drama]
71185       71185  1014_11.png  [Children,  Comedy,  Drama]
71186       71186  1014_12.png  [Children,  Comedy,  Drama]
71187       71187  1014_13.png  [Children,  Comedy,  Drama]

[71188 rows x 3 columns]


In [ ]:
main_genre = [ 'Action','Children','Musical', 'Comedy', 'Drama', 'Fantasy', 'Horror', 'Mystery','Animation', 'Romance', 'Thriller', 'Western', 'Documentary', 'Sci-Fi', 'Adventure', 'Crime']


datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(
    dataframe=df[:40000],
    directory="VisualDataset",
    x_col="ID_Frame",
    y_col="Genres",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    classes=main_genre,
    target_size=(100,100))

valid_generator=test_datagen.flow_from_dataframe(
    dataframe=df[40000:60000],
    directory="VisualDataset",
    x_col="ID_Frame",
    y_col="Genres",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    classes=main_genre,
    target_size=(100,100))

test_generator=test_datagen.flow_from_dataframe(
    dataframe=df[60000:],
    directory="VisualDataset",
    x_col="ID_Frame",
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(100,100))

Found 39896 validated image filenames belonging to 16 classes.
Found 20000 validated image filenames belonging to 16 classes.
Found 11188 validated image filenames.


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='softmax'))
model.compile(optimizers.RMSprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [1]:
checkpoint = ModelCheckpoint("Checkpoints", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks = callbacks_list,
                    class_weight=class_weights,
)

NameError: ignored

In [ ]:


test_generator.reset()
pred=model.predict_generator(test_generator,
    steps=STEP_SIZE_TEST,
    verbose=1)